In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
import data_generator
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from path import col_seq_can, col_seq_can_all
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [5]:
%config Completer.use_jedi = False

### Load Pmpm and IDs

In [6]:
# import boto3
# s3 = boto3.client('s3')
# bucket = 'cdphp-s3-us-e-p-pond'
# path_to_save = '../../../data/PMPM.p'
# s3_path = 'rpi/nneehal/DATA/pmpm_202108_processed_ready.p'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [7]:
# path_to_save = '../../../data/ids.csv'
# s3_path = 'rpi/blind_matching_study/input_data/RPI Test Cohort.csv'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [8]:
%%time
ids_csv = '../../../data/ids.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.79 s, sys: 10.9 s, total: 19.7 s
Wall time: 19.7 s


In [9]:
treated_ids.shape

### Drop Unecessary data points

In [10]:
pmpm_backup = pmpm.copy()

In [11]:
pmpm_backup.MYR.min(), pmpm_backup.MYR.max()

(201701, 202108)

In [12]:
#pmpm_backup2 = pmpm_backup[pmpm_backup.MYR >= 202003].reset_index(drop = True)

In [13]:
pmpm_backup.shape

In [14]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [15]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [16]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [17]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [18]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [19]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [20]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 444 ms, sys: 340 ms, total: 783 ms
Wall time: 783 ms


In [21]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

In [22]:
treated.drop(['MEDICAID_LOB'], axis=1, inplace=True)
controls.drop(['MEDICAID_LOB'], axis=1, inplace=True)

# Take average of treated and controls

### Generate Treated

In [23]:
treated2 = data_process.get_average_record(df=treated, pmpm=pmpm_backup.copy())

PERS_ID
First date
One year before
history from original pmpm
record size calculated
record filtered
returning from f1...
f1 done


100%|██████████| 97/97 [00:00<00:00, 36825.44it/s]

f2 done
stacking done
returning..


In [24]:
treated2.ALLOW_AMT = treated2.ALLOW_AMT.apply(data_process.log_)
treated2.ALLOW_IP = treated2.ALLOW_IP.apply(data_process.log_)
treated2.ALLOW_ER = treated2.ALLOW_ER.apply(data_process.log_)

In [25]:
#treated2.to_pickle("jeremy_avg_treated.p")

### Generate Controls 

In [26]:
controls2 = data_process.get_average_controls_jeremy(controls.copy(), pmpm_backup.copy())

[-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
(158288, 92)
(158288, 77)
[-13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2]
(162237, 92)
(320525, 77)
[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3]
(165840, 92)
(486365, 77)
[-15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4]
(168384, 92)
(654749, 77)
[-16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]
(169947, 92)
(824696, 77)
[-17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6]
(169841, 92)
(994537, 77)


In [27]:
controls2.ALLOW_AMT = controls2.ALLOW_AMT.apply(data_process.log_)
controls2.ALLOW_IP = controls2.ALLOW_IP.apply(data_process.log_)
controls2.ALLOW_ER = controls2.ALLOW_ER.apply(data_process.log_)

In [28]:
#controls2.to_pickle("jeremy_avg_controls.p")

### Save in S3 for reuse

In [29]:
# !aws s3 cp jeremy_avg_treated.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_treated.p --acl bucket-owner-full-control
# !aws s3 cp jeremy_avg_controls.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_controls.p --acl bucket-owner-full-control

In [30]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/get_matches1_8.ipynb s3_get_matches1_8.ipynb --acl bucket-owner-full-control

In [31]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/mapping.xlsx mapping.xlsx --acl bucket-owner-full-control

## Match begin

In [32]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [33]:
def get_data(data_struct, cont):
    cont = cont.copy()
    cont.PERS_ID = cont.PERS_ID.astype(str)
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [34]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [35]:
cols_to_keep = ["PERS_ID", "MYR"]

## Load Model

In [36]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked_logfixed", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [37]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [38]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 12.5 s, sys: 2.69 s, total: 15.1 s
Wall time: 10 s


## Scenario 1 Rigorous

In [39]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:30,  3.25it/s]


CPU times: user 1min 17s, sys: 1min 1s, total: 2min 19s
Wall time: 33.3 s


In [40]:
dat1.shape

## Scenario 2 Non Rigorous

In [41]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:30,  3.22it/s]


CPU times: user 1min 20s, sys: 1min 5s, total: 2min 25s
Wall time: 33.4 s


In [42]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [43]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:54,  1.78it/s]


CPU times: user 1min 51s, sys: 1min 17s, total: 3min 8s
Wall time: 57.7 s


In [44]:
dat3.MYR.min(), dat3.MYR.max()

In [45]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [46]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


17it [00:09,  1.85it/s]


KeyboardInterrupt: 

In [47]:
dat4.shape

NameError: name 'dat4' is not defined

## Scenario 5 Medicaid Rigorous 

In [ ]:
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

In [ ]:
cntrlsM = data_process.transform_features_with_NN(model, controlsM[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

In [ ]:
cntrlsM.PERS_ID = cntrlsM.PERS_ID.astype(str)

In [ ]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st, controls)

In [ ]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [ ]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st, controls)

In [ ]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [ ]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st, controls)

In [ ]:
dat7.MYR.min(), dat7.MYR.max()

In [ ]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [ ]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st, controls)

In [ ]:
dat8.shape

# Save Stats

In [ ]:
from utils import statistics_first_month
import scipy.stats as sts

In [ ]:
columns_to_match = treated.columns.tolist()

In [ ]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



In [ ]:
tr_desc0

In [ ]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
alli[1:]

In [ ]:
Path('./results/stats_mean_std').mkdir(exist_ok = True)
alli[1:].to_csv("./results/stats_mean_std/seq_9_16_mean_std.csv", header=True, index=True)

## Latent Stats

In [ ]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
stat = []
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    stat_a, N, _ = statistics_first_month(dat_lat, latent_cols, return_array = True)
    stat_a = stat_a.T
    stat.append(stat_a)   

In [ ]:
tr_stats, N, _ =  statistics_first_month(tr2, latent_cols, return_array = True)
tr_stats = tr_stats.T

In [ ]:
alli_latent = pd.concat((tr_stats[['mean','std']], stat[0][['mean','std']], stat[1][['mean','std']], stat[2][['mean','std']], stat[3][['mean','std']], 
                  stat[4][['mean','std']], stat[5][['mean','std']], stat[6][['mean','std']], stat[7][['mean','std']]),
                 axis = 1, keys = ['Treated', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7','seq8'])

In [ ]:
alli_latent

In [ ]:
Path('./results/latent_stats_mean_std').mkdir(exist_ok = True)
alli_latent.to_csv("./results/latent_stats_mean_std/seq_9_16_latent_mean_std.csv", index=True, header=True)

## Save latent features

In [ ]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
file_id_start = 9
Path('./results/latent_features').mkdir(exist_ok = True)
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    dat_lat.to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [ ]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq9", 1:"seq10", 2:"seq11", 3:"seq12", 4:"seq13", 5:"seq14", 6:"seq15", 7:"seq16"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [ ]:
ids

In [ ]:
Path('./results/ids').mkdir(exist_ok = True)
ids.to_csv("./results/ids/ids_seq_9_16.csv", index=True, header=True)

## Ratios with/without Std

In [ ]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [ ]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [ ]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [ ]:
std = 0

In [ ]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [ ]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

In [ ]:
d_metric_after_all

In [ ]:
d_metric_before_all

In [ ]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [ ]:
ratio_1_4

In [ ]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [ ]:
mask_1_4

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [ ]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [ ]:
mask_1_4.columns[~mask_1_4.isnull().any()]

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [ ]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [ ]:
mask_1_4.columns[mask_1_4.isnull().all()]

### 1.2 Only Medicaid Controls

In [ ]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

In [ ]:
d_metric_before_med

In [ ]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

In [ ]:
d_metric_after_med

In [ ]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

In [ ]:
ratio_5_8

In [ ]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [ ]:
mask_5_8

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [ ]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [ ]:
mask_5_8.columns[~mask_5_8.isnull().any()]

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [ ]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [ ]:
mask_5_8.columns[mask_5_8.isnull().all()]

### 1.3 Merge and Save ratio

In [ ]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

In [ ]:
Path('./results/ratio').mkdir(exist_ok = True)
final_ratio.to_csv("./results/ratio/ratio_without_std_9_16.csv", index=False, header=True)